You can use this notebook to generate texts using Falcon.

In [ ]:
# clone Lit-GPT
!git clone https://github.com/Lightning-AI/lit-gpt
%cd lit-gpt/

In [ ]:
# for CUDA
!pip install --index-url https://download.pytorch.org/whl/nightly/cu118 --pre 'torch>=2.1.0dev' -q

# install the dependencies
!pip install huggingface_hub -r requirements.txt -q

In [ ]:
# download the weights
!python scripts/download.py --repo_id tiiuae/falcon-7b
!python scripts/convert_hf_checkpoint.py --checkpoint_dir checkpoints/tiiuae/falcon-7b

## Using scripts

In [2]:
# run inference
!python lit_gpt/generate/base.py \
        --prompt "Hello, my name is" \
        --checkpoint_dir checkpoints/tiiuae/falcon-7b \
        --quantize bnb.int8

Loading model '../checkpoints/tiiuae/falcon-7b/lit_model.pth' with {'org': 'Lightning-AI', 'name': 'lit-GPT', 'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': True, 'bias': False, 'n_query_groups': 1, 'shared_attention_norm': True, '_norm_class': 'LayerNorm', 'norm_eps': 1e-05, '_mlp_class': 'GptNeoxMLP', 'intermediate_size': 18176, 'condense_ratio': 1}
bin /home/aniket/miniconda3/envs/parrot/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda121.so
Time to instantiate model: 7.03 seconds.
Time to load the model weights: 21.31 seconds.
Global seed set to 1234
Hello, my name is Jack.
Some people think that dogs are like people. They see them as loyal, loving, intelligent animals. Some people, though, realize that dogs have their own language and their own culture. This is the culture of dog.
Actually,
Time for inference 1: 7.05 sec total, 7.

## Using functional approach


Install lit-gpt as package using `!pip install .`

In [6]:
from lit_gpt.generate.base import build_llm, generate
import lightning as L
import sys
import time
import torch

checkpoint_dir = "checkpoints/tiiuae/falcon-7b"
devices = 1
quantize = "bnb.int8"
max_new_tokens = 50
top_k = 200
temperature = 0.8

In [7]:
model, tokenizer, fabric = build_llm(checkpoint_dir=checkpoint_dir, devices=devices, quantize=quantize)

You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Loading model '../checkpoints/tiiuae/falcon-7b/lit_model.pth' with {'org': 'Lightning-AI', 'name': 'lit-GPT', 'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': True, 'bias': False, 'n_query_groups': 1, 'shared_attention_norm': True, '_norm_class': 'LayerNorm', 'norm_eps': 1e-05, '_mlp_class': 'GptNeoxMLP', 'intermediate_size': 18176, 'condense_ratio': 1}


bin /home/aniket/miniconda3/envs/parrot/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda121.so


Time to instantiate model: 8.95 seconds.
Time to load the model weights: 24.54 seconds.
You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


In [8]:
prompt = "Hello, my name is"
encoded = tokenizer.encode(prompt, device=fabric.device)
prompt_length = encoded.size(0)
max_returned_tokens = prompt_length + max_new_tokens
assert max_returned_tokens <= model.config.block_size, (
    max_returned_tokens,
    model.config.block_size,
)  # maximum rope cache length
L.seed_everything(1234)

Global seed set to 1234


1234

In [9]:
t0 = time.perf_counter()
y = generate(
    model,
    encoded,
    max_returned_tokens,
    max_seq_length=max_returned_tokens,
    temperature=temperature,
    top_k=top_k,
)
t = time.perf_counter() - t0
model.reset_cache()
fabric.print(tokenizer.decode(y))
tokens_generated = y.size(0) - prompt_length
fabric.print(
    f"Time for inference {t:.02f} sec total, {tokens_generated / t:.02f} tokens/sec",
    file=sys.stderr,
)

if fabric.device.type == "cuda":
    fabric.print(
        f"Memory used: {torch.cuda.max_memory_allocated() / 1e9:.02f} GB",
        file=sys.stderr,
    )

Hello, my name is Jack.
Some people think that dogs are like people. They see them as loyal, loving, intelligent animals. Some people, though, realize that dogs have their own language and their own culture. This is the culture of dog.
Actually,


Time for inference 5.79 sec total, 8.63 tokens/sec
Memory used: 8.71 GB
